<a href="https://colab.research.google.com/github/Sillem/suicide_study/blob/main/Projekt_EP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from math import fabs
from sklearn.preprocessing import StandardScaler 

Ustalenie ścieżki do folderu z danymi.

In [2]:
path = r'/content/drive/MyDrive/Projekt EKONOMETRIA PRZESTRZENNA/dane'

Tworzenie słownika z zapisanymi ramkami danych z każdego pliku i zapisywanie ich z podanej ściezki


In [3]:
data = {}

for root, dirs, files in os.walk(path, topdown=False):
   for name in files:
      data[name.replace('.xlsx', '')] = pd.read_excel(os.path.join(root, name)).rename(columns={'Unnamed: 0':'wojewodztwo'})
      data[name.replace('.xlsx', '')]['wojewodztwo'] = data[name.replace('.xlsx', '')]['wojewodztwo'].apply(lambda x: x.replace('WOJ. ', '').rstrip().upper())
      data[name.replace('.xlsx', '')] = data[name.replace('.xlsx', '')].set_index('wojewodztwo').rename(columns={'2018':2018, '2021':2021})

Wyliczanie współczynników zmienności dla każdej zmiennej:

In [4]:
k = {'Współczynnik zmienności':{}}
for key, value in data.items():
  k['Współczynnik zmienności'][key] = f"{round((value.describe().loc['std']/value.describe().loc['mean']).mean() * 100, 2)}%" 

pd.DataFrame(k).sort_values(by='Współczynnik zmienności', ascending=False)

,Współczynnik zmienności
zatrudnieni_zagrozenie,94.37%
rozwody,58.36%
katolicyzm,58.04%
dochody_budzetow_ogolem,54.45%
liczba_samobojstw_na_10_tys,31.93%
stopa_bezrobocia,26.62%
liczba_psychiatrow_na_10_tys,15.94%
osoby_z_wyksztalceniem_podstawowym,15.8%
procent_mezczyzn_w_wojewodztwach,0.78%


Usuwam z badanych zmiennych zmienną procent_mezczyzn_w_wojewodztwach z uwagi na zbyt małą zmienność.

In [5]:
del(data['procent_mezczyzn_w_wojewodztwach'])

Badanie skorelowania zmiennych dla każdego roku. Ale najpierw zbudowanie kompletnego zestawu danych dla każdego roku.

In [6]:
yearly_data = {2010:pd.DataFrame({}), 2014:pd.DataFrame({}), 2018:pd.DataFrame({}), 2021:pd.DataFrame({})}
for key, value in data.items():
  for year in yearly_data.keys():
    yearly_data[year] = pd.concat([yearly_data[year], value[year].rename(key)], axis=1)

((yearly_data[2021].corr() + yearly_data[2018].corr() + yearly_data[2014].corr() + yearly_data[2010].corr())/4).applymap(lambda x: round(x, 3) if fabs(x)>0.8 else 0)

del(data['katolicyzm'])

Usuwam zmienną związaną z katolicyzmem i rozwodami. Usuwam też zmienną objaśniającą.

In [7]:
# del(data['katolicyzm'])
del(data['rozwody'])
del(data['liczba_samobojstw_na_10_tys'])

Sprawdzam korelacje teraz po usunięciu zmiennych.


In [8]:
yearly_data = {2010:pd.DataFrame({}), 2014:pd.DataFrame({}), 2018:pd.DataFrame({}), 2021:pd.DataFrame({})}
for key, value in data.items():
  for year in yearly_data.keys():
    yearly_data[year] = pd.concat([yearly_data[year], value[year].rename(key)], axis=1)

((yearly_data[2021].corr() + yearly_data[2018].corr() + yearly_data[2014].corr() + yearly_data[2010].corr())/4).applymap(lambda x: round(x, 3) if fabs(x)>0.6 else 0)

,dochody_budzetow_ogolem,liczba_psychiatrow_na_10_tys,stopa_bezrobocia,zatrudnieni_zagrozenie,osoby_z_wyksztalceniem_podstawowym
dochody_budzetow_ogolem,1.0,0.0,0.000,0.000,0.000
liczba_psychiatrow_na_10_tys,0.0,1.0,0.000,0.000,0.000
stopa_bezrobocia,0.0,0.0,1.000,-0.612,0.682
zatrudnieni_zagrozenie,0.0,0.0,-0.612,1.000,-0.651
osoby_z_wyksztalceniem_podstawowym,0.0,0.0,0.682,-0.651,1.000


Standaryzacja zmiennych

In [9]:
def standaryzacja_zmiennych(df):
  scaler = StandardScaler()
  return pd.DataFrame(data=scaler.fit_transform(df.values), columns=df.columns, index=df.index)


Funkcja budująca ranking topsis

In [10]:
# tutaj wektor stym_czy_destym będzie definowany jako wektor złożony ze znaków 's' jeżeli kolumna oznacza stymulante
# 'd' jeżeli destymulantę
# dla przykładu dla zemiennych w kolejności stymulanta, destymulanta, stymulanta wektor będzie wyglądał tak ['s', 'd', 's']
def topsis(df, stym_czy_destym):

  def standaryzacja_topsis_zmiennych(df):
    df_copy = df.copy()
    for col in df_copy.columns:
      sqrt_sum_sq = sum(df_copy[col].apply(lambda x: x**2))**0.5
      df_copy[col] = df_copy[col].apply(lambda x: x/sqrt_sum_sq)
    return df_copy
  df = standaryzacja_topsis_zmiennych(df)

  wzorzec = []
  antywzorzec = []
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      wzorzec.append(max(df[col]))
      antywzorzec.append(min(df[col]))
    elif stym_czy_destym[numer] == 'd':
      wzorzec.append(min(df[col]))
      antywzorzec.append(max(df[col]))
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  wzorzec, antywzorzec = np.array(wzorzec), np.array(antywzorzec)
  kopia_operacyjna = df.copy()
  odl_od_wzorca = np.diag(np.matmul((kopia_operacyjna - wzorzec).values, np.transpose((kopia_operacyjna - wzorzec).values)))**0.5
  odl_od_antywzorca = np.diag(np.matmul((kopia_operacyjna - antywzorzec).values, np.transpose((kopia_operacyjna - antywzorzec).values)))**0.5
  topsis = odl_od_antywzorca / (odl_od_wzorca + odl_od_antywzorca)
  ranks = np.argsort(np.argsort(-topsis)) + 1

  return pd.DataFrame(data={'TOPSIS':topsis, 'POZYCJA':ranks}, index=df.index)



Kod do metody sum standaryzowanych

In [11]:
def metoda_sum_std(df, stym_czy_destym):
  """
  Tutaj wymagana jest zewnętrzna funkcja standaryzowania zmiennych z innych komórek,
  nie umieściłem jej wewnątrz tej funkcji żeby móc z niej korzystać przy innych operacjach.
  Przekształcenie topsis umieściłem w funkcji, ponieważ jest ono poza nią bezużyteczne
  """
  df = standaryzacja_zmiennych(df)
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      continue
    elif stym_czy_destym[numer] == 'd':
      df[col] = -df[col] #dla destymulant dajemy przekształconą zmienną z minusem
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  Wi = df.transpose().mean()
  Wi_odj_min = Wi - min(Wi)
  Wi_std = Wi_odj_min / max(Wi_odj_min)
  ranks = np.argsort(np.argsort(-Wi_std)) + 1
  return pd.DataFrame(data={'METODA SUM STANDARYZOWANYCH':Wi_std, 'POZYCJA':ranks}, index=df.index)


# mss = metoda_sum_std(yearly_data[2021], ['d','d','s', 's', 's'])


Kod odpowiedzialny za liczenie rankingu SMR

In [12]:
def smr(df, stym_czy_destym):
  """
  Tutaj wymagana jest zewnętrzna funkcja standaryzowania zmiennych z innych komórek,
  nie umieściłem jej wewnątrz tej funkcji żeby móc z niej korzystać przy innych operacjach.
  Przekształcenie topsis umieściłem w funkcji, ponieważ jest ono poza nią bezużyteczne
  """
  df = standaryzacja_zmiennych(df)
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      continue
    elif stym_czy_destym[numer] == 'd':
      df[col] = -df[col] #dla destymulant dajemy przekształconą zmienną z minusem
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  wzorzec = []
  for col in df.columns:
    wzorzec.append(max(df[col]))
  wzorzec = np.array(wzorzec)
  kopia_operacyjna = df.copy()
  odl_od_wzorca = np.diag(np.matmul((kopia_operacyjna - wzorzec).values, np.transpose((kopia_operacyjna - wzorzec).values)))**0.5
  SMR = 1 - odl_od_wzorca/max(odl_od_wzorca)
  ranks = ranks = np.argsort(np.argsort(-SMR)) + 1
  return pd.DataFrame(data={'METODA SUM STANDARYZOWANYCH':SMR, 'POZYCJA':ranks}, index=df.index)



Teraz przygotowuje kod powiedzialny za wyliczanie rankingu metodą BZW

In [13]:
def BZW(df, stym_czy_destym):
  """
  Tutaj wymagana jest zewnętrzna funkcja standaryzowania zmiennych z innych komórek,
  nie umieściłem jej wewnątrz tej funkcji żeby móc z niej korzystać przy innych operacjach.
  Przekształcenie topsis umieściłem w funkcji, ponieważ jest ono poza nią bezużyteczne
  """
  df = standaryzacja_zmiennych(df)
  if df.columns.shape[0] != len(stym_czy_destym):
    raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
  for numer, col in enumerate(df.columns):
    if stym_czy_destym[numer] == 's':
      continue
    elif stym_czy_destym[numer] == 'd':
      df[col] = -df[col] #dla destymulant dajemy przekształconą zmienną z minusem
    else:
      raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
  kopia_operacyjna = df.copy()
  for col in df.columns:
    kopia_operacyjna[col] = kopia_operacyjna[col] + fabs(min(kopia_operacyjna[col])) 
  max_bzw = sum(kopia_operacyjna.describe().loc['max'].values)
  BZW = kopia_operacyjna.sum(axis=1)/max_bzw
  ranks = ranks = np.argsort(np.argsort(-BZW)) + 1
  return pd.DataFrame(data={'METODA SUM STANDARYZOWANYCH':BZW, 'POZYCJA':ranks}, index=df.index)

Teraz przygotowuje kod odpowiedzialny za wyliczanie rankingu metodą Kukuły

In [14]:
def metoda_kukuly(df, stym_czy_destym):

  def standaryzacja_kukula_zmiennych(df, stym_czy_destym):
    df_copy = df.copy()
    if df.columns.shape[0] != len(stym_czy_destym):
      raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
    for numer, col in enumerate(df.columns):
      if stym_czy_destym[numer] == 's':
        df_copy[col] = (df_copy[col] - min(df_copy[col]))/(max(df_copy[col]) - min(df_copy[col]))
      elif stym_czy_destym[numer] == 'd':
        df_copy[col] = (max(df_copy[col] - df_copy[col]))/(max(df_copy[col]) - min(df_copy[col]))
      else:
        raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
    return df_copy
  df = standaryzacja_kukula_zmiennych(df, stym_czy_destym)

  Qi = df.mean(axis=1)
  ranks = np.argsort(np.argsort(-Qi)) + 1
  return pd.DataFrame(data={'TOPSIS':Qi, 'POZYCJA':ranks}, index=df.index)


kukula = metoda_kukuly(yearly_data[2021], ['d','d','s', 's', 's'])
kukula

,TOPSIS,POZYCJA
DOLNOŚLĄSKIE,0.137334,13
KUJAWSKO-POMORSKIE,0.264866,4
LUBELSKIE,0.292420,3
LUBUSKIE,0.118188,16
ŁÓDZKIE,0.224667,9
MAŁOPOLSKIE,0.141103,11
MAZOWIECKIE,0.121804,15
OPOLSKIE,0.140649,12
PODKARPACKIE,0.299886,2
PODLASKIE,0.256415,6


Tutaj przygotuje kod odpowiedzialny za wyliczanie rankingu metodą Dysonansów

In [15]:
def metoda_dysonansow(df, stym_czy_destym):

  def standaryzacja_metoda_dysonansow(df, stym_czy_destym):
    df_copy = df.copy()
    if df.columns.shape[0] != len(stym_czy_destym):
      raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
    for numer, col in enumerate(df.columns):
      if stym_czy_destym[numer] == 's':
        df_copy[col] = df_copy[col]/max(df_copy[col])
      elif stym_czy_destym[numer] == 'd':
        df_copy[col] = min(df_copy[col])/df_copy[col]
      else:
        raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
    return df_copy
  df = standaryzacja_metoda_dysonansow(df, stym_czy_destym)

  MD = df.mean(axis=1)
  ranks = np.argsort(np.argsort(-MD)) + 1
  return pd.DataFrame(data={'TOPSIS':MD, 'POZYCJA':ranks}, index=df.index)


dysonans = metoda_dysonansow(yearly_data[2021], ['d','d','s', 's', 's'])
dysonans

,TOPSIS,POZYCJA
DOLNOŚLĄSKIE,0.499083,14
KUJAWSKO-POMORSKIE,0.599622,7
LUBELSKIE,0.580755,9
LUBUSKIE,0.655636,4
ŁÓDZKIE,0.529982,12
MAŁOPOLSKIE,0.478702,15
MAZOWIECKIE,0.423906,16
OPOLSKIE,0.658526,3
PODKARPACKIE,0.646367,5
PODLASKIE,0.631621,6


Tutaj przygotuje kod odpowiedzialny za przygotowanie rankingów Metodą Sumy Rang


In [16]:
def metoda_suma_rang(df, stym_czy_destym):

  def standaryzacja_metoda_suma_rang(df, stym_czy_destym):
    df_copy = df.copy()
    if df.columns.shape[0] != len(stym_czy_destym):
      raise ValueError("Liczba elementów wektora określającego charakter zmiennych i liczba kolumn są różne.")
    for numer, col in enumerate(df.columns):
      if stym_czy_destym[numer] == 's':
        df_copy[col] = np.argsort(np.argsort(df_copy[col])) + 1
      elif stym_czy_destym[numer] == 'd':
        df_copy[col] = np.argsort(np.argsort(-df_copy[col])) + 1
      else:
        raise ValueError("Podano inny parametr w wektorze określającym charakter zmiennych niż 's' i 'd'.")
    return df_copy
  df = standaryzacja_metoda_suma_rang(df, stym_czy_destym)

  MSM = df.sum(axis=1)
  ranks = np.argsort(np.argsort(-MSM)) + 1
  return pd.DataFrame(data={'SUMA RANG':MSM, 'POZYCJA':ranks}, index=df.index)


suma_rang = metoda_suma_rang(yearly_data[2021], ['d','d','s', 's', 's'])
suma_rang

,SUMA RANG,POZYCJA
DOLNOŚLĄSKIE,30,14
KUJAWSKO-POMORSKIE,52,4
LUBELSKIE,43,8
LUBUSKIE,50,5
ŁÓDZKIE,40,10
MAŁOPOLSKIE,34,13
MAZOWIECKIE,21,16
OPOLSKIE,48,6
PODKARPACKIE,55,2
PODLASKIE,43,9


Tutaj przygotowuje kod odpowiedzialny za wyliczanie statystyki metody porównywania rankingów Kukuły.

In [18]:
def porownanie_rankingow(pozycje1, pozycje2):
  """
  Funkcja do której przekazujemy dwa obiekty typu pd.Series zawierające pozycje
  przyporządkowane do obiektów. NIE UPORZĄDKOWANE WEDŁUG POZYCJI! Można też podać
  DataFrame z rankingami zbudowanymi na modłę poprzednich przykładów.
  """
  differences = (pozycje1['POZYCJA'] - pozycje2['POZYCJA']).apply(lambda x: fabs(x))
  mianownik = differences.shape[0]**2 - 1 if differences.shape[0]**2 % 1 else differences.shape[0]**2
  return 1- 2*sum(differences)/mianownik




Tutaj definiuje listę metod używanych do użycia (lista funkcji)


In [19]:
metody = [topsis, metoda_sum_std, smr, BZW, metoda_kukuly, metoda_dysonansow, metoda_suma_rang]
rankingi = {x:{} for x in [2010, 2014, 2018, 2021]}
for rok in rankingi.keys():
  for metoda in metody:
    rankingi[rok][metoda.__name__] = metoda(yearly_data[rok], ['d','d','s', 's', 's'])

Tworzę metod porównywania rankingów

In [38]:
from random import randint
def optymalne_rankingi(lista_metod, rankingi):
  def macierz_porownan_rankingow(lista_metod, rankingi):
    macierz = np.identity(len(metody))
    for pierwszy_numer, metoda_pier in enumerate(lista_metod):
      for drugi_numer, metoda_drug in enumerate(lista_metod):
        if pierwszy_numer == drugi_numer: continue
        macierz[pierwszy_numer][drugi_numer] = porownanie_rankingow(rankingi[metoda_pier.__name__], rankingi[metoda_drug.__name__])

    return macierz

  macierz_wynikowa = macierz_porownan_rankingow(metody, rankingi)
  #pd.DataFrame(macierz_wynikowa).to_csv(f"{randint(1,100)}.csv")
  suma = 0
  for col in range(macierz_wynikowa.shape[1]):
    for row in range(macierz_wynikowa.shape[0]):
      if col > row: suma += macierz_wynikowa[row][col]

  srednie_podobienstwo = 2*suma/(len(metody)*(len(metody)-1))
  suma = 0
  for col in range(macierz_wynikowa.shape[1]):
    for row in range(macierz_wynikowa.shape[0]):
      if col > row: suma += fabs(macierz_wynikowa[row][col] - srednie_podobienstwo)

  odchylenie_cwiartkowe_podobienstwo = 2*suma/(len(metody)*(len(metody)-1))
  cutoff = srednie_podobienstwo + odchylenie_cwiartkowe_podobienstwo
  np.fill_diagonal(macierz_wynikowa, 0)
  avg_sim = pd.DataFrame({'średnie podobieństwo':{k.__name__:v for k,v in zip(lista_metod, macierz_wynikowa.mean(axis=1))}})
  avg_sim['średnie podobieństwo'] = avg_sim['średnie podobieństwo'].apply(lambda x: round(x, 3))
  best_ranking_index = np.argmax(macierz_wynikowa.mean(axis=1))
  
  return rankingi[lista_metod[best_ranking_index].__name__].sort_values(by='POZYCJA', ascending=True), avg_sim

pd.concat([optymalne_rankingi(metody, rankingi[rok])[1].rename(columns={'średnie podobieństwo':rok}) for rok in [2010, 2014, 2018, 2021]], axis=1)

,2010,2014,2018,2021
BZW,0.643,0.638,0.712,0.683
metoda_dysonansow,0.592,0.578,0.679,0.618
metoda_kukuly,0.451,0.408,0.583,0.538
metoda_sum_std,0.643,0.638,0.712,0.683
metoda_suma_rang,0.607,0.600,0.679,0.645
smr,0.574,0.576,0.661,0.618
topsis,0.446,0.453,0.516,0.438


Wyliczanie średniego podobieństwa rankingów.

SyntaxError: ignored